In [8]:
import os

In [11]:
os.getcwd()

'e:\\Mlop\\End to end\\RecAPI_MovieFuse'

In [10]:
os.chdir("../")

In [30]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataIngestionConfig:
  base_url: str
  search_endpoint: str
  total_pages: int
  save_file: Path
  CSV_data_path: Path

In [31]:
import sys

In [32]:
from src.RECAPI_MOVIEFUSE.constant import *
from src.RECAPI_MOVIEFUSE.utils.common import read_yaml , create_directories

In [33]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_fetching
        create_directories([Path(config.save_file).parent])
        return DataIngestionConfig(
            base_url=config.base_url,
            search_endpoint=config.search_endpoint,
            total_pages=config.total_pages,
            save_file=Path(config.save_file),
            CSV_data_path=Path(config.CSV_data_path)
        )

In [34]:
import os 
import requests
from src.logger import logging
from src.exception import CustomException
from dotenv import load_dotenv
import time
import json
import pandas as pd

In [35]:
load_dotenv()

True

In [36]:
config=read_yaml(CONFIG_FILE_PATH)
config
# url=f"{config.data_fetching.base_url}{config.data_fetching.search_endpoint}?page={config.data_fetching.total_pages}"
# print(url)

ConfigBox({'data_fetching': {'base_url': 'https://api.themoviedb.org/3', 'search_endpoint': '/discover/movie', 'total_pages': 50, 'save_file': 'artifacts/raw/movies_1000.json', 'CSV_data_path': 'artifacts/CSV_data.csv'}})

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def fetching_data_from_api(self):
        headers = {
            "accept": "application/json",
            "Authorization": f"Bearer {os.getenv('API_KEY')}"
        }

        all_movies = []
        for page in range(1, self.config.total_pages + 1):
            url = f"{self.config.base_url}{self.config.search_endpoint}?page={page}"
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                logging.info(f"Page {page} fetched successfully.")
                data = response.json()
                all_movies.extend(data.get('results', []))
            else:
                logging.warning(f"Failed to fetch page {page}: {response.status_code}")

            time.sleep(0.25)

        with open(self.config.save_file, "w", encoding="utf-8") as f:
            json.dump(all_movies, f, ensure_ascii=False, indent=2)
            
            
            
    def convert_json_to_csv(self):
        with open(self.config.save_file, "r", encoding="utf-8")as f:
            moveis=json.load(f)
            
        processed_movies=[]
        for movie in moveis:
            processed_movies.append({
                "id": movie.get("id"),
                "title": movie.get("title"),
                "original_language": movie.get("original_language"),
                "release_date": movie.get("release_date"),
                "overview": movie.get("overview"),
                "popularity": movie.get("popularity"),
                "vote_average": movie.get("vote_average"),
                "vote_count": movie.get("vote_count"),
                "genre_ids": ",".join(str(gid) for gid in movie.get("genre_ids", []))
            })
            
        df = pd.DataFrame(processed_movies)
        Path(self.config.CSV_data_path).parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(self.config.CSV_data_path, index=False)
        logging.info(f"CSV file saved at {self.config.CSV_data_path}")

In [38]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.fetching_data_from_api()
    data_ingestion.convert_json_to_csv()
    
except Exception as e:
    raise CustomException(e, sys)